<a href="https://colab.research.google.com/github/Akshatha7710/telco-customer-churn/blob/main/Telco_Churn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
"""
Telco Customer Churn - Research-grade end-to-end script (Version A)

Features included:
- Load real Telco Customer Churn CSV (or create synthetic fallback)
- Full EDA (statistics + plots saved to `outputs/figures/`)
- Advanced preprocessing (imputation, encoding, scaling)
- Feature engineering and selection (mutual info + correlation filter)
- Class imbalance handling (SMOTE)
- Multiple models:
    - Logistic Regression (baseline)
    - Decision Tree
    - Random Forest
    - Gradient Boosting (XGBoost if available, else sklearn's HistGradientBoosting)
    - Neural Network (Keras)
- Hyperparameter tuning (RandomizedSearchCV for heavier models)
- Cross-validation and repeated experiments
- Model evaluation: accuracy, precision, recall, f1, ROC-AUC, PR-AUC, confusion matrices
- Model explainability:
    - SHAP explanations (TreeExplainer for tree-based models, KernelExplainer fallback)
    - LIME example (if lime installed)
- Real-time inference function (pipeline)
- Post-deployment considerations: drift detection via PSI and periodic evaluation
- Anomaly scoring via IsolationForest (fraud-like detection)
- Save models, preprocessors, metrics, and artifacts to outputs/
- Safe optional imports (xgboost, lightgbm, shap, lime) handled gracefully

Usage:
- Place `Telco-Customer-Churn.csv` in the `data/` directory (script will create synthetic dataset otherwise).
- Run: python telco_churn_research.py
- Outputs go into `outputs/` directory.

Author: Generated for coursework
"""

import os
import sys
import json
import joblib
import math
import time
import warnings
from pathlib import Path
from typing import Tuple, Dict, Any, List

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import (
    train_test_split, RandomizedSearchCV, GridSearchCV, cross_val_score,
    StratifiedKFold
)
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, roc_auc_score,
    average_precision_score, confusion_matrix, classification_report
)
from sklearn.feature_selection import mutual_info_classif
from sklearn.ensemble import RandomForestClassifier, IsolationForest
from sklearn.tree import DecisionTreeClassifier, export_text
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier

# Optional advanced models
try:
    import xgboost as xgb
    XGBOOST_AVAILABLE = True
except Exception:
    XGBOOST_AVAILABLE = False

try:
    import lightgbm as lgb
    LIGHTGBM_AVAILABLE = True
except Exception:
    LIGHTGBM_AVAILABLE = False

# Neural network
try:
    import tensorflow as tf
    from tensorflow import keras
    from tensorflow.keras import layers
    KERAS_AVAILABLE = True
except Exception:
    KERAS_AVAILABLE = False

# Explainability
try:
    import shap
    SHAP_AVAILABLE = True
except Exception:
    SHAP_AVAILABLE = False

try:
    from lime import lime_tabular
    LIME_AVAILABLE = True
except Exception:
    LIME_AVAILABLE = False

# Imbalance handling
try:
    from imblearn.over_sampling import SMOTE
    IMBLEARN_AVAILABLE = True
except Exception:
    IMBLEARN_AVAILABLE = False

warnings.filterwarnings("ignore")

# ---------------------------
# Configuration & directories
# ---------------------------
ROOT = Path.cwd()
DATA_DIR = ROOT / "data"
OUT_DIR = ROOT / "outputs"
FIG_DIR = OUT_DIR / "figures"
MODEL_DIR = OUT_DIR / "models"
REPORTS_DIR = OUT_DIR / "reports"

for d in (DATA_DIR, OUT_DIR, FIG_DIR, MODEL_DIR, REPORTS_DIR):
    d.mkdir(parents=True, exist_ok=True)

DATA_FILE = DATA_DIR / "Telco-Customer-Churn.csv"
RANDOM_STATE = 42

# ---------------------------
# Utility Functions
# ---------------------------
def save_json(obj: Any, path: Path):
    with open(path, "w") as f:
        json.dump(obj, f, indent=2)

def ensure_df_has_target(df: pd.DataFrame, target: str = "Churn") -> pd.DataFrame:
    if target not in df.columns:
        raise ValueError(f"target column '{target}' not in dataframe")
    return df

def safe_head(df: pd.DataFrame, n=5):
    print(df.head(n).to_string())

# ---------------------------
# Data Loading / Synthetic
# ---------------------------
def create_synthetic_telco(path: Path, n=2000) -> pd.DataFrame:
    rng = np.random.default_rng(RANDOM_STATE)
    df = pd.DataFrame({
        "customerID": [f"CUST{100000+i}" for i in range(n)],
        "gender": rng.choice(["Female", "Male"], n),
        "SeniorCitizen": rng.choice([0,1], n, p=[0.85,0.15]),
        "Partner": rng.choice(["Yes","No"], n, p=[0.45,0.55]),
        "Dependents": rng.choice(["Yes","No"], n, p=[0.25,0.75]),
        "tenure": rng.integers(0, 72, n),
        "PhoneService": rng.choice(["Yes","No"], n, p=[0.9,0.1]),
        "MultipleLines": rng.choice(["Yes","No","No phone service"], n),
        "InternetService": rng.choice(["DSL","Fiber optic","No"], n, p=[0.4,0.45,0.15]),
        "OnlineSecurity": rng.choice(["Yes","No","No internet service"], n),
        "OnlineBackup": rng.choice(["Yes","No","No internet service"], n),
        "DeviceProtection": rng.choice(["Yes","No","No internet service"], n),
        "TechSupport": rng.choice(["Yes","No","No internet service"], n),
        "StreamingTV": rng.choice(["Yes","No","No internet service"], n),
        "StreamingMovies": rng.choice(["Yes","No","No internet service"], n),
        "Contract": rng.choice(["Month-to-month", "One year", "Two year"], n, p=[0.6,0.2,0.2]),
        "PaperlessBilling": rng.choice(["Yes","No"], n),
        "PaymentMethod": rng.choice([
            "Electronic check","Mailed check","Bank transfer (automatic)","Credit card (automatic)"
        ], n),
        "MonthlyCharges": np.round(rng.uniform(18.0, 120.0, n), 2),
    })
    df["TotalCharges"] = np.round(df["MonthlyCharges"] * df["tenure"] + rng.uniform(0, 50, n), 2)
    # churn probability function
    churn_prob = (
        0.35 - 0.004 * df["tenure"] +
        np.where(df["Contract"] == "Month-to-month", 0.2, -0.05) +
        np.where(df["PaymentMethod"] == "Electronic check", 0.05, 0)
    )
    churn_prob = np.clip(churn_prob, 0.01, 0.9)
    df["Churn"] = np.where(rng.random(n) < churn_prob, "Yes", "No")
    df.to_csv(path, index=False)
    return df

def load_data(path: Path = DATA_FILE) -> pd.DataFrame:
    if path.exists():
        df = pd.read_csv(path)
        print(f"[INFO] Loaded dataset from {path} shape={df.shape}")
    else:
        print(f"[WARN] Dataset not found at {path}. Creating synthetic dataset for development.")
        df = create_synthetic_telco(path)
        print(f"[INFO] Synthetic dataset created at {path} shape={df.shape}")
    # normalize column names
    df.columns = [c.strip() for c in df.columns]
    return df

# ---------------------------
# Exploratory Data Analysis
# ---------------------------
def eda_summary(df: pd.DataFrame, target: str = "Churn") -> Dict[str, Any]:
    df = df.copy()
    summary = {}
    summary["shape"] = df.shape
    summary["columns"] = df.columns.tolist()
    summary["dtypes"] = df.dtypes.apply(lambda x: str(x)).to_dict()
    summary["missing"] = df.isnull().sum().to_dict()
    if target in df.columns:
        summary["class_distribution"] = df[target].value_counts().to_dict()
    # numeric stats
    summary["numeric"] = df.select_dtypes(include=[np.number]).describe().to_dict()
    # correlation
    try:
        corr = df.select_dtypes(include=[np.number]).corr()
        summary["correlation"] = corr.fillna(0).to_dict()
    except Exception:
        summary["correlation"] = {}
    # save summary
    save_json(summary, REPORTS_DIR / "eda_summary.json")
    return summary

def eda_plots(df: pd.DataFrame, target: str = "Churn"):
    # churn distribution
    if target in df.columns:
        plt.figure(figsize=(6,4))
        df[target].value_counts().plot(kind="bar")
        plt.title("Churn distribution")
        plt.xticks(rotation=0)
        plt.tight_layout()
        plt.savefig(FIG_DIR / "churn_distribution.png")
        plt.close()

    # Numeric histograms for selected columns
    num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    for c in ["tenure", "MonthlyCharges", "TotalCharges"]:
        if c in df.columns:
            plt.figure(figsize=(6,4))
            df[c].dropna().hist(bins=40)
            plt.title(f"{c} distribution")
            plt.tight_layout()
            plt.savefig(FIG_DIR / f"{c}_hist.png")
            plt.close()

    # Correlation heatmap (numeric)
    try:
        import seaborn as sns
        num = df.select_dtypes(include=[np.number])
        if num.shape[1] > 1:
            plt.figure(figsize=(10,8))
            sns.heatmap(num.corr(), annot=True, fmt=".2f", cmap="coolwarm")
            plt.tight_layout()
            plt.savefig(FIG_DIR / "numeric_corr_heatmap.png")
            plt.close()
    except Exception:
        pass

# ---------------------------
# Preprocessing & Feature Engineering
# ---------------------------
def preprocess(df: pd.DataFrame, target: str = "Churn") -> Tuple[pd.DataFrame, np.ndarray, ColumnTransformer]:
    df = df.copy()
    if "customerID" in df.columns:
        df = df.drop(columns=["customerID"])

    # Convert TotalCharges to numeric (Telco dataset has some spaces)
    if "TotalCharges" in df.columns:
        df["TotalCharges"] = pd.to_numeric(df["TotalCharges"], errors="coerce")

    # Remove rows with missing target
    df = df.dropna(subset=[target])

    # Identify numeric and categorical
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    categorical_cols = df.select_dtypes(include=["object", "category"]).columns.tolist()
    if target in categorical_cols:
        categorical_cols.remove(target)
    if target in numeric_cols:
        numeric_cols.remove(target)

    # Simple feature engineering: tenure bins
    if "tenure" in df.columns:
        df["tenure_bin"] = pd.cut(df["tenure"].fillna(-1), bins=[-1,0,12,24,48,72], labels=["0","1-12","13-24","25-48","49-72"])
        # add to categorical
        categorical_cols.append("tenure_bin")

    # Fill NA in categorical with 'Missing'
    df[categorical_cols] = df[categorical_cols].fillna("Missing")

    # Prepare transformers
    numeric_transformer = Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler())
    ])

    categorical_transformer = Pipeline([
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(handle_unknown="ignore", sparse=False))
    ])

    preprocessor = ColumnTransformer([
        ("num", numeric_transformer, numeric_cols),
        ("cat", categorical_transformer, categorical_cols)
    ], remainder="drop")

    # Label encode target
    y = df[target].apply(lambda x: 1 if str(x).strip().lower() in ["yes", "1", "true", "y"] else 0).values

    X = df.drop(columns=[target]).copy()

    # Save metadata for report
    meta = {
        "numeric_cols": numeric_cols,
        "categorical_cols": categorical_cols,
        "final_shape_before_transform": X.shape
    }
    save_json(meta, REPORTS_DIR / "preprocessing_meta.json")

    return X, y, preprocessor

# ---------------------------
# Feature selection helpers
# ---------------------------
def compute_mutual_info(X: pd.DataFrame, y: np.ndarray, preprocessor: ColumnTransformer, top_k: int = 20) -> List[Tuple[str,float]]:
    # Apply preprocessor to get numeric features (but OneHotEncoder will expand features; we extract names)
    # Fit preprocessor on X to allow get_feature_names_out (sklearn >=1.0)
    preprocessor.fit(X)
    try:
        # Build feature names
        num_cols = []
        cat_cols = []
        for name, trans, cols in preprocessor.transformers_:
            if name == "num":
                num_cols = cols
            if name == "cat":
                cat_cols = cols
        # get transformed feature count & names
        ohe = preprocessor.named_transformers_["cat"].named_steps["onehot"]
        ohe_names = list(ohe.get_feature_names_out(cat_cols))
        feature_names = list(num_cols) + ohe_names
    except Exception:
        # fallback: use numeric columns only
        feature_names = X.select_dtypes(include=[np.number]).columns.tolist()

    # Transform X to numeric matrix for mutual info
    X_num = preprocessor.transform(X)
    mi = mutual_info_classif(X_num, y, random_state=RANDOM_STATE)
    mi_series = pd.Series(mi, index=feature_names).sort_values(ascending=False)
    top = list(mi_series.head(top_k).items())
    # save
    save_json({k: float(v) for k,v in top}, REPORTS_DIR / "mutual_info_top.json")
    return top

# ---------------------------
# Train / Evaluate utilities
# ---------------------------
def classification_metrics(y_true: np.ndarray, y_pred: np.ndarray, y_proba: np.ndarray = None) -> Dict[str, Any]:
    metrics = {}
    metrics["accuracy"] = float(accuracy_score(y_true, y_pred))
    metrics["precision"] = float(precision_score(y_true, y_pred, zero_division=0))
    metrics["recall"] = float(recall_score(y_true, y_pred, zero_division=0))
    metrics["f1"] = float(f1_score(y_true, y_pred, zero_division=0))
    if y_proba is not None:
        try:
            metrics["roc_auc"] = float(roc_auc_score(y_true, y_proba))
        except Exception:
            metrics["roc_auc"] = None
        try:
            metrics["pr_auc"] = float(average_precision_score(y_true, y_proba))
        except Exception:
            metrics["pr_auc"] = None
    else:
        metrics["roc_auc"] = None
        metrics["pr_auc"] = None
    metrics["confusion_matrix"] = confusion_matrix(y_true, y_pred).tolist()
    return metrics

def plot_roc_pr(y_test: np.ndarray, y_proba: np.ndarray, prefix: str):
    # ROC
    try:
        from sklearn.metrics import roc_curve, precision_recall_curve, auc
        fpr, tpr, _ = roc_curve(y_test, y_proba)
        plt.figure(figsize=(6,4))
        plt.plot(fpr, tpr, label=f"AUC={auc(fpr,tpr):.3f}")
        plt.plot([0,1],[0,1],"--", color="gray")
        plt.xlabel("FPR")
        plt.ylabel("TPR")
        plt.title(f"ROC Curve - {prefix}")
        plt.legend()
        plt.tight_layout()
        plt.savefig(FIG_DIR / f"{prefix}_roc.png")
        plt.close()

        precision, recall, _ = precision_recall_curve(y_test, y_proba)
        plt.figure(figsize=(6,4))
        plt.plot(recall, precision)
        plt.xlabel("Recall")
        plt.ylabel("Precision")
        plt.title(f"Precision-Recall Curve - {prefix}")
        plt.tight_layout()
        plt.savefig(FIG_DIR / f"{prefix}_pr.png")
        plt.close()
    except Exception:
        pass

# ---------------------------
# Modeling: training wrappers
# ---------------------------
def train_baselines(X_train, y_train, X_test, y_test, preprocessor):
    # Dummy baseline
    dummy_pipe = Pipeline([("prep", preprocessor), ("clf", DummyClassifier(strategy="most_frequent"))])
    dummy_pipe.fit(X_train, y_train)
    y_pred = dummy_pipe.predict(X_test)
    y_proba = dummy_pipe.predict_proba(X_test)[:,1] if hasattr(dummy_pipe, "predict_proba") else None
    metrics = classification_metrics(y_test, y_pred, y_proba)
    joblib.dump(dummy_pipe, MODEL_DIR / "baseline_dummy.joblib")
    return {"dummy": metrics}

def train_logistic(X_train, y_train, X_test, y_test, preprocessor):
    pipe = Pipeline([("prep", preprocessor), ("clf", LogisticRegression(max_iter=1000, random_state=RANDOM_STATE))])
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)
    y_proba = pipe.predict_proba(X_test)[:,1]
    metrics = classification_metrics(y_test, y_pred, y_proba)
    joblib.dump(pipe, MODEL_DIR / "logistic_pipeline.joblib")
    return {"logistic": metrics, "model": pipe}

def train_random_forest(X_train, y_train, X_test, y_test, preprocessor, random_search=False):
    pipe = Pipeline([("prep", preprocessor), ("clf", RandomForestClassifier(random_state=RANDOM_STATE, n_jobs=-1))])
    if random_search:
        param_dist = {
            "clf__n_estimators": [100, 250, 500],
            "clf__max_depth": [6, 10, 20, None],
            "clf__min_samples_split": [2,5,10],
            "clf__max_features": ["sqrt", "log2", 0.2, 0.5]
        }
        rs = RandomizedSearchCV(pipe, param_distributions=param_dist, n_iter=20, scoring="f1", cv=3, n_jobs=-1, random_state=RANDOM_STATE)
        rs.fit(X_train, y_train)
        best = rs.best_estimator_
    else:
        pipe.fit(X_train, y_train)
        best = pipe
    y_pred = best.predict(X_test)
    y_proba = best.predict_proba(X_test)[:,1]
    metrics = classification_metrics(y_test, y_pred, y_proba)
    joblib.dump(best, MODEL_DIR / "random_forest.joblib")
    return {"random_forest": metrics, "model": best}

def train_xgboost(X_train, y_train, X_test, y_test, preprocessor, random_search=False):
    if not XGBOOST_AVAILABLE:
        print("[WARN] XGBoost not available; skip.")
        return None
    # use sklearn wrapper
    clf = xgb.XGBClassifier(use_label_encoder=False, eval_metric="logloss", random_state=RANDOM_STATE, n_jobs=-1)
    pipe = Pipeline([("prep", preprocessor), ("clf", clf)])
    if random_search:
        param_dist = {
            "clf__n_estimators": [100, 250, 500],
            "clf__max_depth": [3,6,10],
            "clf__learning_rate": [0.01, 0.05, 0.1],
            "clf__subsample": [0.6, 0.8, 1.0]
        }
        rs = RandomizedSearchCV(pipe, param_distributions=param_dist, n_iter=20, scoring="f1", cv=3, n_jobs=-1, random_state=RANDOM_STATE)
        rs.fit(X_train, y_train)
        best = rs.best_estimator_
    else:
        pipe.fit(X_train, y_train)
        best = pipe
    y_pred = best.predict(X_test)
    y_proba = best.predict_proba(X_test)[:,1]
    metrics = classification_metrics(y_test, y_pred, y_proba)
    joblib.dump(best, MODEL_DIR / "xgboost.joblib")
    return {"xgboost": metrics, "model": best}

def train_hist_gbm(X_train, y_train, X_test, y_test, preprocessor):
    # fallback to sklearn's HistGradientBoostingClassifier
    try:
        from sklearn.ensemble import HistGradientBoostingClassifier
        clf = HistGradientBoostingClassifier(random_state=RANDOM_STATE)
        pipe = Pipeline([("prep", preprocessor), ("clf", clf)])
        pipe.fit(X_train, y_train)
        y_pred = pipe.predict(X_test)
        # HGB does not always have predict_proba for some versions; handle gracefully
        y_proba = pipe.predict_proba(X_test)[:,1] if hasattr(pipe, "predict_proba") else None
        metrics = classification_metrics(y_test, y_pred, y_proba)
        joblib.dump(pipe, MODEL_DIR / "hist_gbm.joblib")
        return {"hist_gbm": metrics, "model": pipe}
    except Exception as e:
        print("[WARN] HistGradientBoosting unavailable:", e)
        return None

def train_nn_model(X_train, y_train, X_test, y_test, preprocessor, epochs=30, batch_size=64):
    if not KERAS_AVAILABLE:
        print("[WARN] Keras/TensorFlow not available; skipping NN.")
        return None
    # Preprocess to dense numpy
    X_train_p = preprocessor.fit_transform(X_train)
    X_test_p = preprocessor.transform(X_test)
    input_dim = X_train_p.shape[1]

    model = keras.Sequential([
        layers.Input(shape=(input_dim,)),
        layers.Dense(256, activation="relu"),
        layers.BatchNormalization(),
        layers.Dropout(0.4),
        layers.Dense(128, activation="relu"),
        layers.Dropout(0.3),
        layers.Dense(64, activation="relu"),
        layers.Dense(1, activation="sigmoid")
    ])
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss="binary_crossentropy", metrics=[keras.metrics.AUC(name="auc")])
    early_stop = keras.callbacks.EarlyStopping(monitor="val_auc", mode="max", patience=6, restore_best_weights=True)
    history = model.fit(X_train_p, y_train, validation_split=0.1, epochs=epochs, batch_size=batch_size, callbacks=[early_stop], verbose=0)

    y_proba = model.predict(X_test_p).ravel()
    y_pred = (y_proba >= 0.5).astype(int)
    metrics = classification_metrics(y_test, y_pred, y_proba)

    # Save model and preprocessor
    model_path = MODEL_DIR / "nn_model.keras"
    model.save(model_path)
    joblib.dump(preprocessor, MODEL_DIR / "nn_preprocessor.joblib")
    # Save history
    hist_path = REPORTS_DIR / "nn_history.json"
    save_json({k: [float(v) for v in vals] for k, vals in history.history.items()}, hist_path)

    return {"neural_network": metrics, "model": (model, preprocessor)}

# ---------------------------
# Explainability (SHAP / LIME)
# ---------------------------
def explain_with_shap(model_pipe, X_train: pd.DataFrame, sample_X_test: pd.DataFrame, model_name: str):
    if not SHAP_AVAILABLE:
        print("[WARN] SHAP not installed; install shap to get explanations.")
        return None

    print(f"[INFO] Generating SHAP explanations for {model_name}...")

    # extract the underlying model and preprocessor
    if isinstance(model_pipe, Pipeline):
        pre = model_pipe.named_steps.get("prep")
        clf = model_pipe.named_steps.get("clf")
    else:
        # for (model, preprocessor) tuple
        try:
            clf, pre = model_pipe
        except Exception:
            print("[WARN] Unknown model_pipe structure for SHAP")
            return None

    X_sample_trans = pre.transform(sample_X_test)
    # Use TreeExplainer for tree-based models
    try:
        explainer = shap.Explainer(clf)
        shap_values = explainer(X_sample_trans)
    except Exception:
        # fallback: KernelExplainer (slower)
        explainer = shap.KernelExplainer(lambda x: clf.predict_proba(x)[:,1], shap.sample(X_sample_trans, 100))
        shap_values = explainer.shap_values(X_sample_trans[:100])
    # Save a small summary plot
    try:
        shap.summary_plot(shap_values, X_sample_trans, show=False)
        plt.savefig(FIG_DIR / f"shap_summary_{model_name}.png", bbox_inches="tight")
        plt.close()
    except Exception:
        pass
    return shap_values

def explain_with_lime(model_pipe, X_train: pd.DataFrame, sample_row: pd.Series):
    if not LIME_AVAILABLE:
        print("[WARN] LIME not installed; install lime to get local explanations.")
        return None
    # LIME needs numpy arrays and feature names
    pre = model_pipe.named_steps.get("prep")
    clf = model_pipe.named_steps.get("clf")
    X_train_p = pre.transform(X_train)
    feature_names = []
    # try to get OHE feature names
    try:
        cat_step = pre.named_transformers_["cat"].named_steps["onehot"]
        cat_cols = pre.transformers_[1][2]
        ohe_names = list(cat_step.get_feature_names_out(cat_cols))
        num_cols = pre.transformers_[0][2]
        feature_names = list(num_cols) + ohe_names
    except Exception:
        # fallback
        feature_names = [f"f{i}" for i in range(X_train_p.shape[1])]
    explainer = lime_tabular.LimeTabularExplainer(X_train_p, feature_names=feature_names, class_names=["No","Yes"], discretize_continuous=True)
    row_p = pre.transform(pd.DataFrame([sample_row]))
    exp = explainer.explain_instance(row_p[0], clf.predict_proba, num_features=10)
    return exp

# ---------------------------
# Post-deployment: Drift detection (PSI)
# ---------------------------
def population_stability_index(expected: np.ndarray, actual: np.ndarray, buckets: int = 10) -> float:
    """Compute PSI between expected and actual arrays (numeric)"""
    expected = np.array(expected).ravel()
    actual = np.array(actual).ravel()
    # build quantile bins from expected
    breakpoints = np.percentile(expected, np.linspace(0, 100, buckets+1))
    expected_perc = np.histogram(expected, bins=breakpoints)[0] / len(expected)
    actual_perc = np.histogram(actual, bins=breakpoints)[0] / len(actual)
    # avoid zeros
    eps = 1e-6
    expected_perc = np.where(expected_perc == 0, eps, expected_perc)
    actual_perc = np.where(actual_perc == 0, eps, actual_perc)
    psi = np.sum((expected_perc - actual_perc) * np.log(expected_perc / actual_perc))
    return float(psi)

# ---------------------------
# Anomaly detection (IsolationForest)
# ---------------------------
def build_anomaly_detector(X: pd.DataFrame, preprocessor: ColumnTransformer):
    X_p = preprocessor.fit_transform(X)
    iso = IsolationForest(contamination=0.01, random_state=RANDOM_STATE)
    iso.fit(X_p)
    joblib.dump(iso, MODEL_DIR / "isolation_forest.joblib")
    return iso

def score_anomalies(iso: IsolationForest, X: pd.DataFrame, preprocessor: ColumnTransformer):
    X_p = preprocessor.transform(X)
    scores = -iso.decision_function(X_p)  # higher = more anomalous
    return scores

# ---------------------------
# Real-time inference helper
# ---------------------------
def build_predict_fn(model_pipe) -> callable:
    """
    Returns a function predict_fn(dict_row)->(pred, proba)
    """
    def predict_fn(input_dict: Dict[str, Any]):
        row = pd.DataFrame([input_dict])
        if isinstance(model_pipe, Pipeline):
            proba = model_pipe.predict_proba(row)[:,1]
            pred = (proba >= 0.5).astype(int)
            return int(pred[0]), float(proba[0])
        else:
            # tuple (model, preprocessor) expected for NN
            try:
                model, pre = model_pipe
                Xp = pre.transform(row)
                proba = model.predict(Xp).ravel()
                pred = (proba >= 0.5).astype(int)
                return int(pred[0]), float(proba[0])
            except Exception as e:
                raise RuntimeError("Unknown model_pipe format for prediction") from e

    return predict_fn

# ---------------------------
# Orchestrator: run full experiment
# ---------------------------
def run_full_experiment(random_search: bool = True, balance: bool = True):
    print("[INFO] Loading data")
    df = load_data()
    ensure_df_has_target(df, "Churn")
    eda_summary(df, "Churn")
    eda_plots(df, "Churn")

    print("[INFO] Preprocessing data")
    X, y, preprocessor = preprocess(df, target="Churn")

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=RANDOM_STATE, stratify=y)

    # Handle imbalance with SMOTE if available and requested
    if IMBLEARN_AVAILABLE and balance:
        print("[INFO] Applying SMOTE to training data")
        X_train_prep = preprocessor.fit_transform(X_train)
        sm = SMOTE(random_state=RANDOM_STATE)
        X_res, y_res = sm.fit_resample(X_train_prep, y_train)
        # after resampling, we cannot easily inverse-transform to DataFrame; so for model training we will
        # fit models on pipelines that accept raw X_train and pipeline internal preprocessor will be re-fit.
        # To keep things consistent, we will re-fit pipeline directly on raw X_train but with class_weight if needed.
        use_smote = True
    else:
        X_res, y_res = None, None
        use_smote = False

    results = {}

    # Baseline
    print("[INFO] Training baseline models")
    results.update(train_baselines(X_train, y_train, X_test, y_test, preprocessor))

    # Logistic
    print("[INFO] Training Logistic Regression")
    log_res = train_logistic(X_train, y_train, X_test, y_test, preprocessor)
    results.update({k:v for k,v in log_res.items() if k != "model"})

    # Random Forest
    print("[INFO] Training Random Forest")
    rf_res = train_random_forest(X_train, y_train, X_test, y_test, preprocessor, random_search=random_search)
    results.update({k:v for k,v in rf_res.items() if k != "model"})
    rf_model = rf_res.get("model")

    # XGBoost or HistGradientBoosting
    if XGBOOST_AVAILABLE:
        print("[INFO] Training XGBoost")
        xgb_res = train_xgboost(X_train, y_train, X_test, y_test, preprocessor, random_search=random_search)
        if xgb_res is not None:
            results.update({k:v for k,v in xgb_res.items() if k != "model"})
    else:
        print("[INFO] Training HistGradientBoosting (fallback)")
        hgb_res = train_hist_gbm(X_train, y_train, X_test, y_test, preprocessor)
        if hgb_res:
            results.update({k:v for k,v in hgb_res.items() if k != "model"})

    # Neural Network
    print("[INFO] Training Neural Network (may take a while)")
    nn_res = train_nn_model(X_train, y_train, X_test, y_test, preprocessor, epochs=30, batch_size=128)
    if nn_res:
        results.update({k:v for k,v in nn_res.items() if k != "model"})

    # Compare results and choose best by f1 or roc_auc
    save_json(results, REPORTS_DIR / "model_metrics_summary.json")

    # Example: use random forest pipeline as final model if available
    try:
        final_model = rf_model
    except Exception:
        final_model = None

    if final_model is None:
        print("[WARN] No final model found; using logistic pipeline as final fallback.")
        try:
            final_model = joblib.load(MODEL_DIR / "logistic_pipeline.joblib")
        except Exception:
            final_model = None

    if final_model is not None:
        print("[INFO] Generating explanations for final model (SHAP if available)")
        # pick sample of test set rows for explainability
        sample_X_test = X_test.sample(n=min(200, max(1, len(X_test))), random_state=RANDOM_STATE)
        if SHAP_AVAILABLE:
            try:
                shap_values = explain_with_shap(final_model, X_train, sample_X_test, "final_model")
            except Exception as e:
                print("[WARN] SHAP explanation failed:", e)
        if LIME_AVAILABLE:
            try:
                exp = explain_with_lime(final_model, X_train, sample_X_test.iloc[0])
                # save lime explanation as text
                with open(FIG_DIR / "lime_exp.txt", "w") as f:
                    f.write(str(exp.as_list()))
            except Exception as e:
                print("[WARN] LIME explanation failed:", e)

        # Build predict function and save
        predict_fn = build_predict_fn(final_model)
        # Example prediction saved
        example_row = X_test.iloc[0].to_dict()
        pred, proba = predict_fn(example_row)
        save_json({"example_prediction": {"pred": pred, "proba": proba, "row": example_row}}, REPORTS_DIR / "example_prediction.json")

    # Build anomaly detector on whole training data
    try:
        print("[INFO] Building anomaly detector")
        iso = build_anomaly_detector(X, preprocessor)
        scores = score_anomalies(iso, X_test, preprocessor)
        # attach scores to sample and save
        out_df = X_test.copy()
        out_df["anomaly_score"] = scores
        out_df.head(20).to_csv(REPORTS_DIR / "anomaly_scores_sample.csv", index=False)
    except Exception as e:
        print("[WARN] Anomaly detector failed:", e)

    # Post-deployment: compute PSI between training probability distribution and test probability distribution for a model (if computed)
    try:
        # use logistic model proba if available
        try:
            logistic_pipe = joblib.load(MODEL_DIR / "logistic_pipeline.joblib")
            proba_train = logistic_pipe.predict_proba(X_train)[:,1]
            proba_test = logistic_pipe.predict_proba(X_test)[:,1]
            psi = population_stability_index(proba_train, proba_test, buckets=10)
            save_json({"psi": psi}, REPORTS_DIR / "post_deployment_psi.json")
            print(f"[INFO] PSI computed: {psi:.5f}")
        except Exception:
            pass
    except Exception as e:
        print("[WARN] PSI computation failed:", e)

    print("[INFO] Full experiment complete. Outputs written to:", OUT_DIR)
    return results

# ---------------------------
# Run when executed directly
# ---------------------------
if __name__ == "__main__":
    start_time = time.time()
    results = run_full_experiment(random_search=True, balance=True)
    elapsed = time.time() - start_time
    print(f"[INFO] Total time elapsed: {elapsed/60:.2f} minutes")
    # print brief results
    try:
        print(json.dumps(results, indent=2))
    except Exception:
        pass


[INFO] Loading data
[WARN] Dataset not found at /content/data/Telco-Customer-Churn.csv. Creating synthetic dataset for development.
[INFO] Synthetic dataset created at /content/data/Telco-Customer-Churn.csv shape=(2000, 21)
[INFO] Preprocessing data


TypeError: Cannot setitem on a Categorical with a new category (Missing), set the categories first